In [22]:
import numpy as np
import random
import datetime
import time
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
import pandas as pd
import seaborn as sns

import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
plt.rcParams['font.size'] = 18
#plt.rcParams['font.family'] = "serif"
tdir = 'in'
major = 5.0
minor = 3.0
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor


from ember_utils import *
from ember_model import *
from ember_pjr_utils import *

In [2]:
def get_IFBased_samples(family_name, family_data, num_samples_per_malware_family):
    data_X = np.array(family_data)
    
    if len(data_X) > 1:
        
        # fit the model
        clf = IsolationForest(max_samples=len(data_X))
        clf.fit(data_X)
        #scores_prediction = clf.decision_function(data_X)
        y_pred = clf.predict(data_X)


        anomalous_idx = np.where(y_pred == -1.0)
        similar_idx = np.where(y_pred == 1.0)

        #print(f'{family_name}: all-{len(y_pred)} anomalous-{len(anomalous_idx[0])} similar-{len(similar_idx[0])}')
        assert len(anomalous_idx[0]) + len(similar_idx[0]) == len(y_pred)

        anomalous_samples = data_X[anomalous_idx]
        remaining_samples_to_pick = num_samples_per_malware_family - len(anomalous_samples)
        if abs(remaining_samples_to_pick)/remaining_samples_to_pick == -1:
            remaining_samples_to_pick = min(int(len(anomalous_samples) * 0.50), abs(remaining_samples_to_pick))
        
        if remaining_samples_to_pick >= len(similar_idx):
            similar_samples = data_X[similar_idx]
        else:
            similar_samples_pool = list(data_X[similar_idx])
            similar_samples = random.sample(similar_samples_pool, remaining_samples_to_pick)

        replay_samples = np.concatenate((anomalous_samples, similar_samples))
    else:
        replay_samples = data_X
    
    return replay_samples


def get_IFBased_samples_V1(family_name, family_data, num_samples_per_malware_family):
    data_X = np.array(family_data)
    
    if len(data_X) > 1:
        
        # fit the model
        clf = IsolationForest(max_samples=len(data_X))
        clf.fit(data_X)
        #scores_prediction = clf.decision_function(data_X)
        y_pred = clf.predict(data_X)


        anomalous_idx = np.where(y_pred == -1.0)
        similar_idx = np.where(y_pred == 1.0)

        #print(f'{family_name}: all-{len(y_pred)} anomalous-{len(anomalous_idx[0])} similar-{len(similar_idx[0])}')
        assert len(anomalous_idx[0]) + len(similar_idx[0]) == len(y_pred)

        anomalous_samples = data_X[anomalous_idx]
        remaining_samples_to_pick = num_samples_per_malware_family - len(anomalous_samples)

        if remaining_samples_to_pick >= len(similar_idx):
            similar_samples = data_X[similar_idx]
        else:
            similar_samples_pool = list(data_X[similar_idx])
            similar_samples = random.sample(similar_samples_pool, remaining_samples_to_pick)

        replay_samples = np.concatenate((anomalous_samples, similar_samples))
    else:
        replay_samples = data_X
    
    return replay_samples



def get_replay_samples_IFBased(global_family_dict, num_samples_per_malware_family):
    
    
    tmp_family_dict = defaultdict(list)
    
    pre_malware_samples = []

    cnt = 0
    for k, v in global_family_dict.items():
        if k != 'goodware':
            cnt += 1
            selected_family_samples = get_IFBased_samples(k, v, num_samples_per_malware_family)
            tmp_family_dict[k] = list(selected_family_samples)
            
            #print(selected_family_samples)
            for sample in selected_family_samples:
                pre_malware_samples.append(sample)
                
    if len(global_family_dict['goodware']) < len(pre_malware_samples):
        pre_goodware_samples = random.sample(global_family_dict['goodware'], len(global_family_dict['goodware']))
    else:
        pre_goodware_samples = random.sample(global_family_dict['goodware'], len(pre_malware_samples))
    
    tmp_family_dict['goodware'] = list(pre_goodware_samples)
    
    samples_to_replay = np.concatenate((np.array(pre_goodware_samples), np.array(pre_malware_samples)))
    labels_to_replay = np.concatenate((np.zeros(len(pre_goodware_samples)), np.ones(len(pre_malware_samples))))


    #print(f'X_replay {samples_to_replay.shape} Y_replay {labels_to_replay.shape}')
    #print(f'Replay {len(pre_malware_samples)} malware samples of {len(global_family_dict.keys()) -1} families')
    #print(f'and Replay {len(pre_goodware_samples)} goodware samples')
    
    
    return samples_to_replay, labels_to_replay, tmp_family_dict

In [23]:
def training_final_score(model, X_, Y_, batch_size, device):
    
    testloader = get_dataloader(X_test, Y_test, batch_size, train_data=False)   
    
    model.eval()
    y_pred_list = []
    y_true_list = []
    
    
    with torch.no_grad():
        for x_batch, y_batch in tqdm(testloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            y_pred_list += list(y_test_pred.squeeze(1).cpu().numpy())
    
            y_true_list += list(y_batch.cpu().numpy())
        
    assert len(y_true_list) == len(y_pred_list) 
    return np.array(y_pred_list), np.array(y_true_list)

In [ ]:
all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

data_dir = '../../month_based_processing_with_family_labels/'



patience = 5
replay_type = 'partial_joint_replay'



num_exps = 1 #args.num_exps
#task_month = args.task_month
num_epoch = 500 #args.num_epoch
batch_size = 6000 #args.batch_size
#replay_portion = args.replay_portion
num_samples_per_malware_family = 500

exp_type = 'ifbased_' #{'', 'last', 'random', 'ifbased'}

exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]


allexps_acc = {}
allexps_rocauc = {}
allexps_training_time = {}
all_exps_best_epoch = {}

mistaken_stats = {}

cnt =  1    
for exp in exp_seeds:
    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net()
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    

    
    stored_global_family_dict = defaultdict(list)
    
    standardization = StandardScaler()
    standard_scaler = None
    for task_month in range(len(all_task_months)):
                
        print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Round {cnt} ...')
        task_start = time.time()
        
        #task_month = task_month
        current_task = all_task_months[task_month]
        task_months = all_task_months[:task_month+1]
        print(f'Current Task {current_task} w/ {num_samples_per_malware_family} samples to Replay per Malware family.')


        model_save_dir = '../IFBased_pjr_saved_model_' +\
                    str(exp_type) + '/IFBased_PJR_replay_' +\
                    str(num_samples_per_malware_family) + '/' + str(current_task) + '/'
        create_parent_folder(model_save_dir)
        
        opt_save_path = '../IFBased_pjr_saved_optimizer_' +\
                    str(exp_type) + '/IFBased_PJR_replay_' +\
                    str(num_samples_per_malware_family) + '/' + str(current_task) + '/'
        create_parent_folder(opt_save_path)
        
        
        results_save_dir = './IFBased_saved_results_' +\
                    str(exp_type) + '/IFBased_PJR_replay_' +\
                    str(num_samples_per_malware_family) + '/' 
        create_parent_folder(results_save_dir)

        
        X_train, Y_train, Y_train_family = get_family_labeled_month_data(data_dir, current_task)
        X_test, Y_test, Y_test_family = get_family_labeled_task_test_data(data_dir, task_months, mlp_net=True)
        
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
        standard_scaler = standardization.partial_fit(X_train)

        X_train = standard_scaler.transform(X_train)
        X_test = standard_scaler.transform(X_test)

        X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
        X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
        
        
        # to debug
        #X_train, Y_train, Y_train_family = X_train[:500], Y_train [:500], Y_train_family[:500]
        #X_test, Y_test, Y_test_family = X_test[:50], Y_test[:50], Y_test_family[:50]
        

        if current_task == all_task_months[0]:
            num_replay_samples = 0
            stored_global_family_dict = make_family_based_dict(X_train, Y_train, Y_train_family,\
                                                                   current_task, stored_global_family_dict)
        else:
            num_replay_samples = sum([len(v) for k, v in stored_global_family_dict.items()])
            print(f'replay samples of task {num_replay_samples}')
            
    
            stored_global_family_dict = make_family_based_dict(X_train, Y_train, Y_train_family,\
                                                                   current_task, stored_global_family_dict)
            X_train, Y_train, stored_global_family_dict =\
                get_replay_samples_IFBased(stored_global_family_dict, num_samples_per_malware_family)
            
        print()
        print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
        print()
        
        '''
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
        standard_scaler = standardization.partial_fit(X_train)

        X_train = standard_scaler.transform(X_train)
        X_test = standard_scaler.transform(X_test)

        X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
        X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
        '''        
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Training ...')
        task_training_time, epoch_ran, training_loss, validation_loss  =\
                                training_early_stopping(model, model_save_dir, opt_save_path,\
                                X_train, Y_train, X_test, Y_test, patience,\
                                batch_size, device, optimizer, num_epoch,\
                                 criterion, replay_type, current_task, exp, earlystopping=True)
        
        
        
        model = Ember_MLP_Net()
        model = model.to(device)
        #load the best model for this task
        best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
        print(f'loading best model {best_model_path}')
        model.load_state_dict(torch.load(best_model_path))
        
        
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
        best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
        print(f'loading best optimizer {best_optimizer}')
        optimizer.load_state_dict(torch.load(best_optimizer))
        
        
        
        y_preds, y_trues = training_final_score(model, X_train, Y_train, batch_size, device)
        
        
        plt.figure(figsize=(8, 6))

        save_file_name = str(task_month + 1) + '_trainingSampleScores.png'
        sns.scatterplot(x=y_preds, y=[i for i in range(len(y_preds))],\
                        marker='+',  alpha=1.0, color='red')

        plt.savefig('./scores_fig/' + save_file_name, bbox_inches='tight', dpi=1000);

        
        
        acc, rocauc = testing_aucscore(model, X_test, Y_test, batch_size, device)
        
        
        end_time = time.time()

        print(f'Elapsed time {(end_time - start_time)/60} mins.')    
        

        task_end = time.time()
        task_run_time = (task_end - task_start)/60
        
        
        try:
            allexps_acc[str(current_task)].append(acc)
            allexps_rocauc[str(current_task)].append(rocauc)
            allexps_training_time[str(current_task)].append(task_run_time)
            all_exps_best_epoch[str(current_task)].append(epoch_ran)
        except:
            allexps_acc[str(current_task)] = [acc]
            allexps_rocauc[str(current_task)] = [rocauc]
            allexps_training_time[str(current_task)] = [task_run_time]
            all_exps_best_epoch[str(current_task)] = [epoch_ran]
        
        

        results_f = open(os.path.join(results_save_dir + '011023_ifnext2_' + str(num_samples_per_malware_family) + '_results.txt'), 'a')
        result_string = '{}\t{}\t{}\t{}\t{}\t{}\t\n'.format(current_task,epoch_ran, task_training_time, acc, rocauc, num_replay_samples)
        results_f.write(result_string)
        results_f.flush()
        results_f.close()
        
    
    end_time = time.time()
    cnt += 1
    print(f'Elapsed time {(end_time - start_time)/60} mins.')
   


Torch 1.6.0 CUDA 10.2
Model has 3.132161m parameters

2023-01-10 22:11:28 Round 1 ...
Current Task 2018-01 w/ 500 samples to Replay per Malware family.
X_train (55722, 2381) Y_train (55722,) Y_tr_family (55722,)
X_test (6192, 2381) Y_test (6192,) Y_te_family (6192,)
2023-01-10 22:11:30 Standardizing ...


  0%|          | 0/9 [00:00<?, ?it/s]

Task 2018-01 and #-of new samples stored 55722

X_train (55722, 2381) Y_train (55722,)

2023-01-10 22:11:33 Training ...
Epoch 1 of 500


  0%|          | 0/9 [00:00<?, ?it/s]

Train Loss: 0.7089, Train Acc: 0.5226
Val Loss: 0.6844, Val Acc: 0.5590
Validation loss decreased (inf --> 0.684360).  Saving model ...
../IFBased_pjr_saved_model_ifbased_/IFBased_PJR_replay_500/2018-01/best_model_epoch_1.pt
../IFBased_pjr_saved_optimizer_ifbased_/IFBased_PJR_replay_500/2018-01/best_optimizer_epoch_1.pt
Epoch 2 of 500


...

  0%|          | 0/97 [00:00<?, ?it/s]

Train Loss: 0.0839, Train Acc: 0.9691
Val Loss: 0.1156, Val Acc: 0.9578
EarlyStopping counter: 1 out of 5
Epoch 3 of 500


  0%|          | 0/97 [00:00<?, ?it/s]

Train Loss: 0.0811, Train Acc: 0.9697
Val Loss: 0.1238, Val Acc: 0.9521
EarlyStopping counter: 2 out of 5
Epoch 4 of 500


 68%|██████▊   | 66/97 [00:10<00:03,  7.97it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

